# Controls Application Analysis

Here we dump metrics from all applications, console, java, etc. and collect them in a single data frame. This allows us to explore the data and discover insights that would be difficult to know otherwise.

## Single Source of Truth

We will keep all data in the `app_data.json` file. So, if you generate data that could be used for future analysis, add it to the `app_data` data frame and then write the data frame back to the `app_data.json` file.

```python
# Example data concatenation
# app_data = pd.concat([app_data, line_data])
app_data.to_json("app_data.json", orient="records", indent=2)
```

### Output to JSON

If you uncomment this, it will output to `test.json`. Use this as a guide to update the `app_data.json` file. Check the changes with a print or the `test.json` file before committing changes to `app_data.json`.

In [ ]:
# app_data = pd.read_json('app_data.json')
# app_data = app_data.merge(line_data, on="program")
# app_data.to_json('test.json', orient='records', indent=2)

## Imports

To install these you can run `python3 -m pip install -r requirements.txt`.

In [ ]:
import json

import pandas as pd
# Even though we don't use this, it has useful defaults, so let's keep it.
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
app_data = pd.read_json('app_data.json')
java_data = pd.read_json('java_data.json')

# We will want to filter by whether or not an application is mapped (NaN)
# this only applies for pas

# Filter by exection_count > 0 for pas & sas

# Define logical conditions to cut application from data
# count -> lines of code
# execution count -> number of times application is opened 
nonobsolete_condition = app_data["status"] == "active"
nonzero_lines_condition = app_data["count"]>0
nonzero_execution_count_condition = app_data["execution_count"]>0
mapped_condition = app_data["index_page"].notnull()

# Logical AND all the conditions
# cut_condition = nonobsolete_condition*nonzero_lines_condition*nonzero_execution_count_condition*mapped_condition
cut_condition = nonobsolete_condition*nonzero_lines_condition
print(f'{np.where(cut_condition)[0].size} elements made the cut out of {np.asarray(app_data["count"]).size}.')

lines = np.asarray(app_data['count'])[np.where(cut_condition)]
exect_count = np.asarray(app_data['execution_count'])[np.where(cut_condition)]

In [ ]:
# Getting some stats
pas = app_data.loc[app_data['type'] == 'pas']
sas = app_data.loc[app_data['type'] == 'sas']
uls = app_data.loc[app_data['type'] == 'uls']
# Total number of active PAs and SAs
actives = app_data.loc[app_data['status'] == 'active']
active_pas = actives.loc[actives['type'] == 'pas']
active_sas = actives.loc[actives['type'] == 'sas']
# Total number of active ULs
active_uls = actives.loc[actives['type'] == 'uls']

# lines of code sum
sum_pa = active_pas['count'].sum()
sum_sa = active_sas['count'].sum()
sum_ul = active_uls['count'].sum()

keeper_count  = actives.groupby('keeper').agg(['count']).stack()
mapped_pas = active_pas.loc[active_pas['index_page'].notnull()]

# criticalitity counts
core = app_data.loc[app_data['criticality'] == 'core']
critical = app_data.loc[app_data['criticality'] == 'critical']
specialized = app_data.loc[app_data['criticality'] == 'specialized']

# All index page mappings
map_list = active_pas['index_page'].apply(pd.Series).stack().unique()
machine = []
for app in map_list :
    machine.append(app[0])
unique_machines = pd.DataFrame(machine)
unique_machines = unique_machines.value_counts()

plt.hist(machine, 15, label="Mapping by Index Page")
plt.legend()
plt.show()

In [ ]:
# Historgram of Keepers
keepers = actives[['keeper']].values

plt.hist(keepers, 70, label="keeper occurances")
plt.legend()

# locs, labels = plt.xticks()
# plt.setp(labels, rotation=90)
plt.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
plt.show()

In [ ]:
# mapping 
map_count = active_pas['index_page'].str.len()
map_count_sum = map_count.sum()
active_pas['map_count'] = active_pas['index_page'].str.len()

plt.hist(map_count, 90, label="Mapping Count")
plt.legend()

plt.show()

In [ ]:
# Apply fit to data
def func(x, a, b, c):
    return a * np.exp(-1.0*(x/b)) + c # Exponential decay

In [ ]:
# Histogram of execution count

x = np.linspace(0, exect_count.size - 1, exect_count.size)
nbins = 100

n, bin_borders, _ = plt.hist(exect_count, nbins, label="Execution Count")
bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
p0 = [1, 1E2, 1]
# popt, pcov = curve_fit(func, bin_centers, n, p0)
# plt.plot(bin_centers, func(bin_centers, *popt), 'r-', label="Exponential Fit")
#plt.xlim(0,50000)
plt.gca().set_yscale('log')
cutoff = 5 # units of fit characteristic decay constant
# plt.axvline(x = cutoff*popt[1], ymin = 0, ymax = 1, color='green', label = f"Cutoff = {cutoff}"+r"$\lambda$")
plt.legend()

# print(f"Decay constant is {popt[1]}")

# Compute total lines of code within the cutoff
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
# Find index that is nearest to the cutoff, integrate number of lines up to cutoff
# Note from Adam: I don't think this is working yet.
#cutoff_index = np.where(lines == find_nearest(lines, func(x,*popt)))[0][0]
#lines_at_cutoff = np.sum(lines[0:cutoff_index])
#print(f"Total lines of code up to cutoff is {lines_at_cutoff}")

In [ ]:
# Same as above but for liines of code

x = np.linspace(0, lines.size - 1, lines.size)
nbins = 100

n, bin_borders, _ = plt.hist(lines, nbins, label="Lines of Code")
bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
p0 = [1, 1E2, 1]
# popt, pcov = curve_fit(func, bin_centers, n, p0)
# plt.plot(bin_centers, func(bin_centers, *popt), 'r-', label="Exponential Fit")
#plt.xlim(0,50000)
plt.gca().set_yscale('log')
cutoff = 5 # units of fit characteristic decay constant
# plt.axvline(x = cutoff*popt[1], ymin = 0, ymax = 1, color='green', label = f"Cutoff = {cutoff}"+r"$\lambda$")
plt.legend()

# print(f"Decay constant is {popt[1]}")

# Compute total lines of code within the cutoff
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
# Find index that is nearest to the cutoff, integrate number of lines up to cutoff
# Note from Adam: I don't think this is working yet.
#cutoff_index = np.where(lines == find_nearest(lines, func(x,*popt)))[0][0]
#lines_at_cutoff = np.sum(lines[0:cutoff_index])
#print(f"Total lines of code up to cutoff is {lines_at_cutoff}")

In [ ]:
# Some statistics to see what's interesting
mean = np.mean(lines)
median = np.median(lines)
mode = stats.mode(lines)[0]
sum = np.sum(lines)
max = np.max(lines)
print(f"Mean: {mean:.0f}, Median: {median:.0f}, Mode: {mode:.0f}, Sum: {sum:.0f}, Max: {max}")

In [ ]:
# Determine PA# nearest to median
median_index = np.where(lines == find_nearest(lines, np.median(lines)))[0][0]
app_data["program"][median_index]

#### Ideas:
- Remove unused applications from dataset, if possible (i.e. not mapped to index page, not frequently used over time frame, Linac)
- Weight applications by priority (usage stats and interview criticality), redo analysis to determine window
- Use exponential fit, choose windows as integer multiples fit characteristic time constant (instead of standard deviation)

Maybe for CD-1, do first and third idea above. Priority weighting can refine estimate for CD-2.

John: effort is not linear with lines-of-code. Really wants to see how long it takes to port median application.

## Estimation Calculations

Functions below include assumptions and equations for quickly calculating estimates based on lines of code (loc).


In [ ]:
parameter_page_loc = app_data[app_data["program"] == "pa0052"]["count"] # pa0052 is the parameter page id
# The parameter page took .25 FTEs to reach minimal viable product (MVP)
ftes_to_mvp = .25
# Each 204.05 WBS has 5 FTEs available
available_ftes = 5
# An FTE is 52 weeks
fte_weeks = 52
# While our iterations are 4 weeks, we only get 3 weeks of work to allow a week for catch-up and education
iteration_weeks = 4
iteration_work_weeks = 3
# Available FTEs per week
ftes_per_week = available_ftes/fte_weeks
# We have 3 weeks of work per iteration
work_per_iteration = ftes_per_week*iteration_work_weeks

def loc_to_ftes(loc):
    return loc*ftes_to_mvp/parameter_page_loc

def ftes_to_iterations(ftes):
    return ftes/work_per_iteration

def loc_to_iterations(loc):
    return ftes_to_iterations(loc_to_ftes(loc))


In [ ]:
app_data['ftes'] = app_data['count'].apply(loc_to_ftes)
app_data['iterations'] = app_data['count'].apply(loc_to_iterations)

## Dependencies

`get_dependencies.py` generates `dependencies.json`.

`dependencies.json` has the reported dependencies of each property by reading the the associated `depends.opt` in the project directory in mecca.

In [ ]:
# Load the JSON data
with open('dependencies.json', 'r') as fp:
   data = json.load(fp)

# Preprocess the JSON data
preprocessed_data = []
for project, deps in data.items():
   for dep in deps:
       preprocessed_data.append({'project': project, 'dependency': dep})

# Load the preprocessed data into a DataFrame
dependencies = pd.DataFrame(preprocessed_data)
grouped_dependencies = dependencies.groupby('project')
no_dependencies = grouped_dependencies.filter(lambda x: len(x) == 0)
counts = dependencies['dependency'].value_counts()

In [ ]:
# Recursively track dependencies to see if there are circular dependencies
def find_cycle(project, visited, stack, path, dependencies):
    # Mark the current node as visited and add it to the stack
    visited.add(project)
    stack.add(project)
    path.append(project)

    # Recur for all dependencies of this project
    for dep in dependencies[dependencies['project'] == project]['dependency']:
        # print(dep)
        # If the dependency is a project, and is not visited, then recur on it
        # if dep in dependencies['project'] and dep not in visited:
        if dependencies['project'].str.contains(dep).any() and dep not in visited:
            cycle = find_cycle(dep, visited, stack, path, dependencies)
            if cycle:
                return cycle
        # If the dependency is in the stack, then it has a cycle
        elif dep in stack:
            # Get the index of the dependency in the path
            index = path.index(dep)
            # Return the cycle from the path
            return path[index:]

    # Remove the project from the stack as it finishes
    stack.remove(project)
    path.pop()

    return None

def find_interdependencies(dependencies):
    # Create a set to store visited projects and a stack
    visited = set()
    stack = set()
    path = []

    # Call the recursive function to detect cycle in different DFS trees
    for project in dependencies['project']:
        if project not in visited:
            cycle = find_cycle(project, visited, stack, path, dependencies)
            if cycle:
                return cycle

    return None

In [ ]:
find_interdependencies(dependencies)

In [ ]:
# Recursively find all dependencies of a project
def find_dependencies(project, visited, dependencies):
    # Mark the current node as visited and add it to the stack
    visited.add(project)

    # Recur for all dependencies of this project
    for dep in dependencies[dependencies['project'] == project]['dependency']:
        # If the dependency is a project, and is not visited, then recur on it
        if dependencies['project'].str.contains(dep).any() and dep not in visited:
            find_dependencies(dep, visited, dependencies)

In [ ]:
found_dependencies = set()
find_dependencies('acld', found_dependencies, dependencies)
print(found_dependencies)

In [ ]:
beau = pd.read_json('app_data_scope_beau.json')
guz = pd.read_json('app_data_scope_guzman.json')

common_programs = beau['program'][beau['program'].isin(guz['program'])]
uncommon_programs_beau = beau[~beau['program'].isin(guz['program'])]
uncommon_programs_guzman = guz[~guz['program'].isin(beau['program'])]

print(f'overlap: ${len(common_programs)}')
print(f'guz unique: ${len(uncommon_programs_guzman)}')
print(f'beau unique: ${len(uncommon_programs_beau)}')

In [ ]:
app_data[app_data['status'] == 'active'][~app_data['sqa_level'].isin(['unknown', 'Low', 'Medium'])]

In [ ]:
active_core = app_data[app_data['status'] == 'active'][app_data['criticality'] == 'core']
active_critical = app_data[app_data['status'] == 'active'][app_data['criticality'] == 'critical']
active_specialized = app_data[app_data['status'] == 'active'][app_data['criticality'] == 'specialized']

# sum the iterations
sum_iterations = app_data['iterations'].sum()
sum_core_iterations = active_core['iterations'].sum()
sum_critical_iterations = active_critical['iterations'].sum()
sum_specialized_iterations = active_specialized['iterations'].sum()

print(sum_iterations)
print(sum_core_iterations)
print(sum_critical_iterations)
print(sum_specialized_iterations)
print(len(active_critical))

## Establish cuts for CD-1

This is the strategy for determining which applications and support services are included in the project effort through FY29.

In [ ]:
import re

active_with_loc_cut = nonobsolete_condition*nonzero_lines_condition
aggressive_cut = nonobsolete_condition*nonzero_lines_condition*nonzero_execution_count_condition*mapped_condition
aggressive_cut2 = nonobsolete_condition*nonzero_lines_condition*nonzero_execution_count_condition
# test_pages = app_data['index_page'].str.contains('W|X|Z')
# non_test_pages = ~app_data['index_page'].str.contains('[A-Za-z]', na=False)
# test_index_page_cut = aggressive_cut & (app_data['index_page'].str.contains('W|X|Z')) & (~app_data['index_page'].str.contains('[A-Za-z]', na=False))
# non_test_cut_test = app_data['index_page'].str.contains('W')
# test_pages = app_data['index_page'].map(lambda x: 'W' in x or 'X' in x or 'Z' in x)
# test_pages = app_data['index_page'].str.contains('[WXZ]', na=True)
# non_test_pages = app_data['index_page'].str.contains('[A-Z]', na=True)
# non_test_pages = app_data['index_page'].map(lambda x: bool(re.match('[A-Z]', x)))
# non_test_cut_test = app_data['index_page'].dropna().str.contains('W|X|Z')
# non_test_cut_non = ~app_data['index_page'].str.contains('[A-Za-z]', na=False)
# non_test_cut = test_pages & (~app_data['index_page'].str.contains('[A-Za-z]', na=False))
# test_index_page_cut = test_pages & ~non_test_pages
print(f'{np.where(cut_condition)[0].size} elements made the cut out of {np.asarray(app_data["count"]).size}.')

# active_with_loc = app_data[active_with_loc_cut]
# aggressive = app_data[aggressive_cut]
# non_test = app_data[test_index_page_cut]
# print(aggressive)
# print(test_index_page_cut)

def is_test_page(value):
    if value is None:
        return False
    else:
        for page in value:
            if re.match('[A-VY]\d{1,3}', page):
                return True

        return False

non_test_pages = app_data['index_page'].transform(is_test_page)
blah = app_data[non_test_pages]
print(blah['iterations'].sum())
# print(non_test_cut_non)
# print(non_test_cut)
# print(aggressive_cut2)

In [ ]:
app_data["line_usage"] = app_data["execution_count"]/app_data["count"]
app_data["usage_rank"] = app_data["execution_count"].rank(method='dense', ascending=False)
app_data["line_rank"] = app_data["count"].rank(method='dense', ascending=False)
app_data["line_usage_rank"] = app_data["line_usage"].rank(method='dense', ascending=False)
app_data["line_and_usage_rank"] = app_data["usage_rank"] + app_data["line_rank"]



In [ ]:
app_data["count"].max()

In [ ]:
app_data[app_data["status"] == "active"]["count"].sum()

In [ ]:
from numpy import NaN

naive = app_data[app_data["status"] == "active"]["iterations"].sum()/2/12

manual_inclusion = [
    "ul_acl",
    "pa0001", # Index page
    "ul_dabbel",
    "ul_acld_protocol",
    "ul_drf2",
    "ul_scopes",
    "ul_clib",
    "ul_bpmtbt",
    "ul_cbsaux",
    "ul_mecarlib",
    "ul_model",
    "ul_multiwire",
    "ul_nova",
    "ul_pa1516",
    "ul_pasa",
    "ul_pccns",
    "ul_physlib",
    "ul_calc_device",
    "ul_lma",
    "ul_lssol",
    "ul_vacuum",
    "ul_waller",
    "pa4240", # This is a service to launch registered ACL scripts.
    "pa2041", # This program provides support services to the Sequencer.
    "pa0095", # Curve Fit II program which fits data to exps or polynomials
    "pa4138", # This program provides various ACL support utilities.
    "pa1881", # This application launches ACL scripts
    "pa1535", # This is a tool for Fourier signal analysis.
    "pa1661", # Multicast alarms. (The alarm screen)
    "pa4234", # Downtime logger
]
audited_lines_of_code = {
    "ul_acl": 744_228,
    "ul_clib": 200_000,
    "pa4283": 2_069,
    "ul_cbsaux": 269_084,
    "pa1048": 76_521,
    "pa4247": 0,
    "pa1860": 0,
}

for inclusion in manual_inclusion:
    app_data.loc[app_data["program"] == inclusion, "criticality"] = "manual"

# Copy count column to new "audited_loc" column
app_data["audited_loc"] = app_data.loc[:, "count"]

# TODO: Manually adjust LOC
for loc in audited_lines_of_code:
    app_data.loc[app_data["program"] == loc, "audited_loc"] = audited_lines_of_code[loc]

app_data["audited_loc_ftes"] = app_data["audited_loc"].apply(loc_to_ftes)

apps_to_modernize = app_data[app_data["status"] == "active"][app_data["criticality"].notna()]
not_to_modernize = app_data[app_data["status"] == "active"][~app_data["criticality"].notna()]
not_to_modernize.to_csv("not_to_modernize.csv")
apps_to_modernize["audited_loc_ftes"].sum()
# app_data[app_data["criticality"] == "specialized"].count()

In [ ]:
plots = app_data[app_data["status"] == "active"][app_data["description"].str.contains("plot", case=False, na=False)]
plots_filtered = plots[~plots["description"].str.contains("vac|loss|blm|multiwire|tlg|cdf|downtime|tev|waterfall|tune|tbt|mwire|torpedo|example|damper|oddmod|bpm|scope|beam|bs events|lecroy|timeline|histogram|MiniBoone|wirescan|oxygen", case=False, na=False)]
iterations = plots_filtered["iterations"].sum()

print(f'{iterations}')
print(f'{iterations*5}')
print(plots_filtered)

In [ ]:
active_critical.count()


In [ ]:
core_apps = app_data[app_data["criticality"] == "core"]["description"]
critical_apps = app_data[app_data["criticality"] == "critical"]["description"]
specialized_apps = app_data[app_data["criticality"] == "specialized"]["description"]
core_apps.to_json('core_apps.json', orient='records', indent=2)
critical_apps.to_json('critical_apps.json', orient='records', indent=2)
specialized_apps.to_json('specialized_apps.json', orient='records', indent=2)

In [ ]:
core_apps